<a href="https://colab.research.google.com/github/arindamkr/YBI_Internship/blob/main/Food_Meal_Order.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from datetime import datetime, date

meals = []
meal_ids = set()
next_id = 1

def validate_date(date_str: str) -> date:
    """Validate future or today's date in YYYY-MM-DD format."""
    d = datetime.strptime(date_str, "%Y-%m-%d").date()
    if d < date.today():
        raise ValueError("Date cannot be in the past.")
    return d

def add_meal(name: str, date_str: str, ingredients_str: str) -> int:
    """Add a meal with validated date and list of ingredients."""
    global next_id
    d = validate_date(date_str)
    ingredients = [i.strip() for i in ingredients_str.split(",") if i.strip()]
    if not ingredients:
        raise ValueError("No ingredients provided.")
    mid = next_id
    next_id += 1
    meals.append({
        "id": mid,
        "name": name,
        "date": d,
        "ingredients": ingredients
    })
    meal_ids.add(mid)
    print(f"✅ Added meal #{mid}: \"{name}\" on {d} with {len(ingredients)} ingredient(s).")
    return mid

def view_meals():
    """List all meals sorted by date."""
    if not meals:
        print("No meals scheduled yet.")
        return
    print("\n📅 Scheduled Meals:")
    for m in sorted(meals, key=lambda x: x["date"]):
        ingr = ", ".join(m["ingredients"])
        print(f"  {m['id']}: {m['name']} — {m['date']} — Ingredients: {ingr}")
    print()

def generate_ingredients_df() -> pd.DataFrame:
    """Aggregate ingredients across all meals into count summary."""
    data = [
        {"ingredient": ing, "meal_id": m["id"]}
        for m in meals for ing in m["ingredients"]
    ]
    if not data:
        return pd.DataFrame(columns=["ingredient", "count"])
    df = pd.DataFrame(data)
    return df.groupby("ingredient").size().reset_index(name="count")

def export_ingredients_csv(filename="ingredients.csv"):
    """Export aggregated ingredient counts to CSV."""
    df = generate_ingredients_df()
    if df.empty:
        print("⚠️ No ingredients to export.")
        return
    df.to_csv(filename, index=False)
    print(f"✅ Exported {len(df)} ingredients to {filename}")

def main():
    print("🥗 HealthyBites Weekly Meal Scheduler\n")
    while True:
        cmd = input("Choose an action [add/view/export/quit]: ").strip().lower()
        try:
            if cmd == "add":
                name = input("  Meal name: ").strip()
                date_str = input("  Date (YYYY-MM-DD): ").strip()
                ingr = input("  Ingredients (comma-separated): ").strip()
                add_meal(name, date_str, ingr)
            elif cmd == "view":
                view_meals()
            elif cmd == "export":
                export_ingredients_csv()
            elif cmd == "quit":
                print("Goodbye 👋")
                break
            else:
                print("Unknown command. Try again.")
        except Exception as e:
            print(f"⚠️ Error: {e}")

if __name__ == "__main__":
    main()